# Optimizing the number of 'good' neurons in a 1D pool

Calibrator does a lot of this now with `optimize_for_yield()`, but the process was developed in this notebook.

We'll compare results near the end. 


In [ ]:
%load_ext autoreload
%autoreload 2

from pystorm.hal import HAL
from pystorm.PyDriver import bddriver as bd
from pystorm.hal.net_builder import NetBuilder
from pystorm.hal.calibrator import Calibrator, PoolSpec

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np

In [ ]:
np.random.seed(1)

# making a Y-by-X pool of D dims located at (LY, LX)

Y, X = (16, 16)
LY, LX = (16, 0)

N = X * Y
D = 1 # has to be 1 for this

SY = Y // 2
SX = X // 2

In [ ]:
hal = HAL()
net_builder = NetBuilder(hal)
cal = Calibrator(hal)

ps = PoolSpec(YX=(Y,X), loc_yx=(LY, LX), D=1)

nrn_tap_matrix, syn_tap_matrix = cal.create_optimized_yx_taps(ps)
ps.TPM = nrn_tap_matrix

In [ ]:
# determine if syn_tap_matrix is right/left or up/down +1/-1 
# so I know if I'm cutting it in two correctly

plt.figure()
plt.imshow(syn_tap_matrix.reshape(SY, SX))
plt.colorbar()
plt.title('tap point assignments\ncorner coordinates:\nCCW from top-left: (0, 0), (SY, 0), (SY, SX), (0, SX)')

print(syn_tap_matrix[0, 0])
print(syn_tap_matrix[SY-1, 0])
print(syn_tap_matrix[0, SX-1])
print(syn_tap_matrix[SY-1, SX-1])

In [ ]:
# set up the Network object, call yield optimization function
def run_network(ps_orig, diff_G=1024, diff_R=1024, cut='none', biases=0):
    ps = ps_orig.copy()
    ps.biases = biases
    
    DAC_vals = {
        'DAC_SOMA_REF': 1024,
        'DAC_DIFF_G': diff_G,
        'DAC_DIFF_R': diff_R}

    # net is now mapped, try slicing the diffusor
    if cut == 'line':
        ps.diffusor_cuts_yx = NetBuilder.get_diff_cuts_to_break_pool_in_half(ps.Y, ps.X)

    # determine fmax
    ps.fmax = cal.optimize_fmax(ps)

    # estimate encoders and offsets
    encoders, offsets, _, _ = cal.get_encoders_and_offsets(ps, dacs=DAC_vals,
                                                           num_sample_angles=3,
                                                           solver='scipy_opt')
    
    return encoders, offsets, ps, DAC_vals

In [ ]:
# do exhaustive twiddle val search, like Terry does
# TODO come back and try with the calibration

import pickle
pck_fname = 'raw_offsets.pck'
REDO_SWEEP = False

if REDO_SWEEP:
    raw_offsets = np.zeros((7, N))
    for bias_idx, bias in enumerate([-3, -2, -1, 0, 1, 2, 3]):
        encoders, offsets, _, _ = run_network(ps, diff_G=1024, diff_R=1024, cut='line', biases=bias)
        raw_offsets[bias_idx, :] = offsets
    pickle.dump(raw_offsets, open(pck_fname, 'wb'))
else:
    raw_offsets = pickle.load(open(pck_fname, 'rb'))
    
# make it like the soma_bias_twiddle calibration, relative to bias twiddle 0
all_offsets = raw_offsets.copy()
orig_offsets_at_3 = raw_offsets[6, :]
orig_offsets_at_0 = raw_offsets[3, :]
    
# subtract out 0 bias
for bias_idx, bias in enumerate([-3, -2, -1, 0, 1, 2, 3]):
    all_offsets[bias_idx, :] -= orig_offsets_at_0

In [ ]:
# plot resulting offsets
plt.figure()
plt.plot(raw_offsets)

plt.figure()
plt.plot(all_offsets)

# estimate slope of each neurons sampled twiddle levels
# the mismatch of each level is correlated
# this is a better way of estimating unsampled bias levels than what 
# the calibration does

all_offsets_est = Calibrator.extrapolate_bias_twiddles(all_offsets)

plt.figure()
plt.plot(all_offsets_est)
print('hi')


In [ ]:
def optimize_twiddles_once(ps, cut, diff_G, diff_R):

    def run_curr_network(biases):
        return run_network(ps, diff_G=diff_G, diff_R=diff_R, cut=cut, biases=biases)

    encoders, offsets, _, _ = run_curr_network(3)
    offsets_at_3 = offsets

    bias_settings, new_offsets, good, bin_counts, dbg = \
        Calibrator.optimize_bias_twiddles(encoders, offsets_at_3, all_offsets_est, policy='center')

    fs = (15, 15)
    xylim = (0, 800, -800, 1500)
    Calibrator.plot_neuron_yield_cone(encoders, new_offsets, good,
                                     (encoders, offsets, bias_settings),
                                      title='expected',
                                      figsize=fs,
                                      xylim=xylim)

    encoders_opt, offsets_opt, ps_opt, dacs_opt = run_curr_network(bias_settings)

    Calibrator.plot_neuron_yield_cone(encoders_opt, offsets_opt, good,
                                     (encoders, offsets, bias_settings),
                                      title='trick opt vs trick',
                                      figsize=fs,
                                      xylim=xylim)

    good_orig = np.sum(Calibrator.get_good_mask(encoders, offsets))
    good_exp = np.sum(Calibrator.get_good_mask(encoders, new_offsets))
    good_ver = np.sum(Calibrator.get_good_mask(encoders_opt, offsets_opt))
    print('good orig:', good_orig)
    print('good exp:', good_exp)
    print('good ver:', good_ver)
    
    return encoders_opt, offsets_opt, good_ver, ps_opt, dacs_opt
        
def do_validation_exp(encoders_opt, offsets_opt, ps, dacs):
    ps.fmax = cal.optimize_fmax(ps)
    NUM_VAL_SAMPLES = 20
    val_pts = np.linspace(-1, 1, NUM_VAL_SAMPLES).reshape((NUM_VAL_SAMPLES, 1))
    rmse, meas_A, est_A = cal.validate_est_encs(encoders_opt, offsets_opt, ps, val_pts, dacs=dacs)
    return val_pts, meas_A

def plot_validation_exp(encoders_opt, offsets_opt, val_pts, meas_A, lines_per_plot=None):

    clean_encs = encoders_opt.copy()
    unest = np.isnan(offsets_opt)
    clean_encs[unest, :] = 0
    clean_offsets = offsets_opt.copy()
    clean_offsets[unest] = 0

    est_A = np.maximum(0, np.dot(val_pts, clean_encs.T) + clean_offsets)

    if lines_per_plot is not None:
        n_neurons = meas_A.shape[1]
        plot_nrn_idxs = np.random.permutation(np.arange(n_neurons))[:lines_per_plot]
        print(plot_nrn_idxs)
        meas_A_plot = meas_A[:, plot_nrn_idxs]
        est_A_plot = est_A[:, plot_nrn_idxs]
    else:
        meas_A_plot = meas_A
        est_A_plot = est_A
        
    plt.figure(figsize=(15,15))
    plt.gca().set_prop_cycle(None)
    plt.plot(val_pts, meas_A_plot, '.-')
    plt.gca().set_prop_cycle(None)
    plt.plot(val_pts, est_A_plot, '-')
    plt.axis([-1, 1, 0, 1500]) 
    plt.title('estimated vs measured tuning curves')
    
    tuned_on = encoders_opt[:, 0].flatten() > 0
    meas_A_on = meas_A[:, tuned_on]
    meas_A_off = meas_A[:, ~tuned_on]
    est_A_on = est_A[:, tuned_on]
    est_A_off = est_A[:, ~tuned_on]
    
    fig, ax = plt.subplots(2, 2, figsize=(15, 15))
    ax[0, 0].plot(val_pts, meas_A_off)
    ax[0, 1].plot(val_pts, meas_A_on)
    ax[0, 0].axis([-1, 1, 0, 1500]) 
    ax[0, 1].axis([-1, 1, 0, 1500]) 
    ax[1, 0].plot(val_pts, est_A_off)
    ax[1, 1].plot(val_pts, est_A_on)
    ax[1, 0].axis([-1, 1, 0, 1500]) 
    ax[1, 1].axis([-1, 1, 0, 1500]) 
    plt.suptitle('measured (above) and estimated (below) tuning curves, N = 256, .5s of raw spikes data/pt')
    
    plt.figure(figsize=(15,12))
    plt.plot(val_pts, est_A)
    plt.axis([-1, 1, 0, 1500]) 
    plt.title('estimated tuning curves')

    plt.figure()
    is_est = ~unest
    gains = Calibrator.get_gains(encoders_opt[is_est, :])
    intercepts = -offsets_opt[is_est] / gains
    plt.hist(intercepts, bins=np.linspace(-2, 2, 21))
    plt.title('intercept distribution')
        

In [ ]:
encs, offs, good_ct, ps_opt, dacs_opt = optimize_twiddles_once(ps, 'line', 1024, 1024)

In [ ]:
val_pts, meas_A = do_validation_exp(encs, offs, ps_opt, dacs_opt)

In [ ]:
plot_validation_exp(encs, offs, val_pts, meas_A, lines_per_plot=50)

# Compare to Calibrator.optimize_yield()

In [ ]:
opt_ps_in = PoolSpec(YX=(Y,X), loc_yx=(LY, LX), D=2)
DAC_vals = {
        'DAC_SOMA_REF': 1024,
        'DAC_DIFF_G': 1024,
        'DAC_DIFF_R': 1024}

opt_ps_out, opt_encs, opt_offsets, dbg = \
    cal.optimize_yield(opt_ps_in, dacs=DAC_vals, bias_twiddle_policy='center', offset_source='calibration_db')

In [ ]:
before_encs, before_offsets = dbg['before']
exp_encs, exp_offsets = dbg['expected']

opt_good = Calibrator.get_good_mask(opt_encs, opt_offsets)
exp_good = Calibrator.get_good_mask(exp_encs, exp_offsets)

print('good exp:', np.sum(exp_good))
print('good ver:', np.sum(opt_good))


fs = (10, 10)
xylim = (0, 800, -800, 1500)

Calibrator.plot_neuron_yield_cone(exp_encs, exp_offsets, exp_good,
                                 (before_encs, before_offsets, opt_ps_out.biases),
                                  title='expected',
                                  figsize=fs,
                                  xylim=xylim)

Calibrator.plot_neuron_yield_cone(opt_encs, opt_offsets, exp_good,
                                 (before_encs, before_offsets, opt_ps_out.biases),
                                  title='actual',
                                  figsize=fs,
                                  xylim=xylim)



# Diffusor Hyperparameter Sweeps

In [ ]:
# sweep diffusor spread, measuring for yield
def run_diffusor_sweep(ps, cut, diff_Gs, diff_Rs):
    encs = []
    offs = []
    good_cts = []
    for diff_G, diff_R in zip(diff_Gs, diff_Rs):
        optimize_twiddles_once(ps, cut, diff_G, diff_R)
        
        encs.append(encoders_opt)
        offs.append(encoders_opt)
        good_cts.append(good_ver)
        
    return encs, offs, good_cts, net_opt
        

In [ ]:
diff_Rs = [500, 1024, 1024]
diff_Gs = [1024, 1024, 500]
cut = 'line'
run_diffusor_sweep(ps, cut, diff_Gs, diff_Rs)


In [ ]:
diff_Rs = [1024, 500, 100]
diff_Gs = [1024]*len(diff_Rs)
cut = 'none'
run_diffusor_sweep(ps, cut, diff_Gs, diff_Rs)

In [ ]:
# see how much bad_syns helps

default_syn_tap_matrix = NetBuilder.create_default_yx_taps(Y // 2, X // 2, D)
default_nrn_tap_matrix = NetBuilder.syn_taps_to_nrn_taps(default_syn_tap_matrix)
NetBuilder.make_taps_even(default_nrn_tap_matrix)

ps_bad_taps = ps.copy()

diff_Rs = [1024]
diff_Gs = [1024]

cut = 'line'
run_diffusor_sweep(ps_bad_taps, cut, diff_Gs, diff_Rs)